# Including Bus Routes

Purpose: Identify bus stop locations along routes in order to estimate the vehicle drive cycle.

Work Flow: Isolate bus stops along desired route. Calculate distance between route points and bus stops. 

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

In [2]:
import route_dynamics.base as base

import geopandas as gpd
import pandas as pd
import branca.colormap as cm
import folium
import rasterstats
import matplotlib.pyplot as plt

from folium.features import GeoJson
from shapely.geometry import mapping
from shapely.geometry import LineString
from shapely.geometry import Polygon
from rasterio.mask import mask
from geopy.distance import geodesic

First, input data and select bus route. 

In [3]:
routes_shp = '../data/six_routes.shp'
stops_shp = '../data/Transit_Stops_for_King_County_Metro__transitstop_point.shp'
route_num = 45

Use base functions, `read_shape` and `extract_point_df` to create a dataframe of route points.  

In [4]:
route = base.read_shape(routes_shp, route_num)
points = base.extract_point_df(route)

Isolate bus stops that service desired route number.

In [5]:
stops = gpd.read_file(stops_shp)

First, need to remove some pesky 'none' types...

In [6]:
stops['ROUTE_LIST'].fillna(value=str(0), inplace=True)

In [7]:
stops_list = pd.DataFrame()
for i in range(0, len(stops)):
    
    if str(route_num) in (stops['ROUTE_LIST'][i]):
        for x in stops['ROUTE_LIST'][i].split(' '):
            if str(route_num) == x:
                stops_list = stops_list.append(stops.iloc[i])
            else:
                pass
        #stops_list = pd.concat([stops_list, stops.iloc[i]], axis = 0)
    else:
        pass


In [9]:
stop_ids = stops_list['STOP_ID'].values

Create a dataframe of bus stop points.

In [10]:
geometry = stops_list.geometry.values

xy = []
for i in range(len(geometry)):
    dic = mapping(geometry[i])
    coords = dic['coordinates']
    xy.append(coords)
    xy_df = pd.DataFrame(columns = ['STOP_ID','coordinates'])
    xy_df['coordinates'] = xy
    
xy_df['STOP_ID'] = stops_list['STOP_ID'].values
xy_df.head()

,STOP_ID,coordinates
0,5330.0,"(-122.35493491168019, 47.69052891200809)"
1,5350.0,"(-122.34960617887032, 47.6905094486964)"
2,5370.0,"(-122.34472418112374, 47.69049011915158)"
3,5380.0,"(-122.34150318558568, 47.69047109565094)"
4,5400.0,"(-122.33660476604088, 47.6904450549253)"


In [20]:
def make_lines(gdf, idx):
    
    coordinate_1 = gdf.loc[idx]['coordinates']
    coordinate_2 = gdf.loc[idx + 1]['coordinates']
    line = LineString([coordinate_1, coordinate_2])
    data = {'geometry':[line]}
    df_line = pd.DataFrame(data, columns = ['geometry'])

    return df_line


def make_route_lines(gdf):
    df_route = pd.DataFrame(columns = ['geometry'])
    for idx in range(len(gdf) - 1):
        df_linestring = make_lines(gdf, idx)
        df_route = pd.concat([df_route, df_linestring])
        
    gdf_route = gpd.GeoDataFrame(df_route)
    
    return gdf_route

In [21]:
gdf_route = make_route_lines(points)

In [22]:
gdf_stops = pd.DataFrame(xy, columns = ['Lat','Long'])

Show map

In [46]:
UW_coords = [47.655548, -122.303200]
figure_size = folium.Figure(height = 400)
route_map = folium.Map(location = UW_coords, zoom_start = 12)
route_json = gdf_route.to_json()
route_layer = folium.GeoJson(route_json)
route_layer.add_child
route_map.add_child(route_layer)
route_map.add_to(figure_size)

fg = folium.FeatureGroup()
for i in range(len(gdf_stops)):
    fg.add_child(folium.Circle(location=[gdf_stops['Long'][i],gdf_stops['Lat'][i]], radius=8, color='red'))

route_map.add_child(fg)
route_map